In [ ]:
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Dropout, LSTM, Concatenate, Input, Bidirectional, RepeatVector, Reshape, GaussianNoise
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import regularizers
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras.models import Model
import random
import math
import scipy as sp
import numpy as np
import scipy.io
import cmath
from math import log
from google.colab import drive 
from numpy import savetxt
drive.mount('/content/gdrive')
################################################################################

Xcu_train = np.load('/content/gdrive/My Drive/Xcu_train64x.npy')
Xcx_train = np.load('/content/gdrive/My Drive/Xcx_train64x.npy')
Xcy_train = np.load('/content/gdrive/My Drive/Xcy_train64x.npy')
Xcz_train = np.load('/content/gdrive/My Drive/Xcz_train64x.npy')
Xcu_test = np.load('/content/gdrive/My Drive/Xcu_test64x.npy')
Xcx_test = np.load('/content/gdrive/My Drive/Xcx_test64x.npy')
Xcy_test = np.load('/content/gdrive/My Drive/Xcy_test64x.npy')
Xcz_test = np.load('/content/gdrive/My Drive/Xcz_test64x.npy')
Yc_train = np.load('/content/gdrive/My Drive/Yc_train64x.npy')
Yc_test = np.load('/content/gdrive/My Drive/Yc_val64x.npy')
###############
Xcx_trainx = np.load('/content/gdrive/My Drive/Xcx_train15x.npy')
Xcy_trainx = np.load('/content/gdrive/My Drive/Xcy_train15x.npy')
Xcz_trainx = np.load('/content/gdrive/My Drive/Xcz_train15x.npy')
Xcx_testx = np.load('/content/gdrive/My Drive/Xcx_test15x.npy')
Xcy_testx = np.load('/content/gdrive/My Drive/Xcy_test15x.npy')
Xcz_testx = np.load('/content/gdrive/My Drive/Xcz_test15x.npy')
Yc_trainx = np.load('/content/gdrive/My Drive/Yc_train15x.npy')
Yc_testx = np.load('/content/gdrive/My Drive/Yc_val15x.npy')
################################################################################
###### Creating CNN model ######################################################
################################################################################
################################################################################
L2 = 0.001
N = 64
M = 4
towers = []
inp_u = Input(shape=(N*M,1))
inp_x = Input(shape=(N*M,1))
inp_y = Input(shape=(N*M,1))
inp_z = Input(shape=(N*M,1))
#
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_u)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_cnn = Dense(128, kernel_regularizer=regularizers.l2(L2), activity_regularizer=regularizers.l2(L2))(fc_1_drop)
towers.append(out_cnn)
u = Model(inputs=inp_u, outputs=out_cnn)
#
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_x)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_cnn = Dense(128, kernel_regularizer=regularizers.l2(L2), activity_regularizer=regularizers.l2(L2))(fc_1_drop)
towers.append(out_cnn)
x = Model(inputs=inp_x, outputs=out_cnn)
#
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_y)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_cnn = Dense(128, kernel_regularizer=regularizers.l2(L2), activity_regularizer=regularizers.l2(L2))(fc_1_drop)
towers.append(out_cnn)
y = Model(inputs=inp_y, outputs=out_cnn)
#
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_z)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_cnn = Dense(128, kernel_regularizer=regularizers.l2(L2), activity_regularizer=regularizers.l2(L2))(fc_1_drop)
towers.append(out_cnn)
z = Model(inputs=inp_z, outputs=out_cnn)
#
out = Concatenate()(towers)
out = RepeatVector(1)(out)
d = LSTM(64, dropout=0.1, return_sequences=False)(out)
d = Reshape((64,))(d)
d = Dense(64, kernel_regularizer=regularizers.l2(L2), activity_regularizer=regularizers.l2(L2))(d)
d = Dropout(0.1)(d)
d = Dense(2, activation='softmax')(d)
cnnl_model = Model(inputs=[inp_u, inp_x, inp_y, inp_z], outputs=d)
cnnl_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
history_CNNL = cnnl_model.fit([Xcu_train, Xcx_train, Xcy_train, Xcz_train], Yc_train, batch_size=100, epochs=5, verbose=1, validation_data=([Xcu_test, Xcx_test, Xcy_test, Xcz_test], Yc_test))
cnnl_model.save('/content/gdrive/My Drive/CNN_LSTM.h5')
################################################################################
towers = []
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_u)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_u = LSTM(64, dropout=0.1, return_sequences=False)(fc_1_reshape)
towers.append(out_u)
u = Model(inputs=inp_u, outputs=out_u)
###
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_x)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_x = LSTM(64, dropout=0.1, return_sequences=False)(fc_1_reshape)
towers.append(out_x)
x = Model(inputs=inp_x, outputs=out_x)
###
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_y)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_y = LSTM(64, dropout=0.1, return_sequences=False)(fc_1_reshape)
towers.append(out_y)
y = Model(inputs=inp_y, outputs=out_y)
###
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_z)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_z = LSTM(64, dropout=0.1, return_sequences=False)(fc_1_reshape)
towers.append(out_z)
z = Model(inputs=inp_z, outputs=out_z)
#
out = Concatenate()(towers)
out = Reshape((256,))(out)
out = Dense(128, kernel_regularizer=regularizers.l2(L2), activity_regularizer=regularizers.l2(L2))(out)
d = Dropout(0.1)(out)
out = Dense(64)(d)
out = Dropout(0.1)(out)
d_out = Dense(2, activation='softmax')(out)
cnnblstm_model = Model(inputs=[inp_u, inp_x, inp_y, inp_z], outputs=d_out)
cnnblstm_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
history_CNNBLSTM = cnnblstm_model.fit([Xcu_train, Xcx_train, Xcy_train, Xcz_train], Yc_train, batch_size=100, epochs=5, verbose=1, validation_data=([Xcu_test, Xcx_test, Xcy_test, Xcz_test], Yc_test))
cnnblstm_model.save('/content/gdrive/My Drive/CNN_LSTM_MLP.h5')
################################################################################
towers = []
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_u)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_u = Bidirectional(LSTM(64, dropout=0.1, return_sequences=False))(fc_1_reshape)
towers.append(out_u)
u = Model(inputs=inp_u, outputs=out_u)
###
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_x)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_x = Bidirectional(LSTM(64, dropout=0.1, return_sequences=False))(fc_1_reshape)
towers.append(out_x)
x = Model(inputs=inp_x, outputs=out_x)
###
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_y)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_y = Bidirectional(LSTM(64, dropout=0.1, return_sequences=False))(fc_1_reshape)
towers.append(out_y)
y = Model(inputs=inp_y, outputs=out_y)
###
c1 = Conv1D(40, 9, padding='same', strides=1)(inp_z)
c1_activation = Activation('relu')(c1)
conv_2 = Conv1D(40, 9, padding='same', strides=1)(c1_activation)
conv_2_activation = Activation('relu')(conv_2)
max = MaxPooling1D(pool_size=2)(conv_2_activation)
fc_1_drop = Flatten()(max)
fc_1_drop = Dropout(0.1)(fc_1_drop)
fc_1_reshape = RepeatVector(1)(fc_1_drop)
out_z = Bidirectional(LSTM(64, dropout=0.1, return_sequences=False))(fc_1_reshape)
towers.append(out_z)
z = Model(inputs=inp_z, outputs=out_z)
#
out = Concatenate()(towers)
out = Reshape((512,))(out)
out = Dense(128, kernel_regularizer=regularizers.l2(L2), activity_regularizer=regularizers.l2(L2))(out)
d = Dropout(0.1)(out)
out = Dense(64)(d)
out = Dropout(0.1)(out)
d_out = Dense(2, activation='softmax')(out)
cnnblstm_model = Model(inputs=[inp_u, inp_x, inp_y, inp_z], outputs=d_out)
cnnblstm_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
history_CNNBLSTM = cnnblstm_model.fit([Xcu_train, Xcx_train, Xcy_train, Xcz_train], Yc_train, batch_size=100, epochs=5, verbose=1, validation_data=([Xcu_test, Xcx_test, Xcy_test, Xcz_test], Yc_test))
cnnblstm_model.save('/content/gdrive/My Drive/CNN_BLSTM_MLP.h5')
################################################################################
inp_x = Input(shape=(15,15,1))
inp_y = Input(shape=(15,15,1))
inp_z = Input(shape=(15,15,1))
towers = []
heads = []
c1 = Conv2D(30, (5,5), padding='same', activation='relu', strides=1)(inp_x)
conv_2 = Conv2D(20, (5,5), padding='same', activation='relu', strides=1)(c1)
max = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same')(conv_2)
fc_1 = Flatten()(max)
out = Dense(256)(fc_1)
heads.append(out)
fc_1_drop = RepeatVector(1)(out)
out_lstm = LSTM(64, return_sequences=False)(fc_1_drop)
out_lstm = Reshape((64,))(out_lstm)
heads.append(out_lstm)
out_cnnlstm= Concatenate()(heads)
towers.append(out_cnnlstm)
x = Model(inputs=inp_x, outputs=out_cnnlstm)
#
heads = []
c1 = Conv2D(30, (5,5), padding='same', activation='relu', strides=1)(inp_y)
conv_2 = Conv2D(20, (5,5), padding='same', activation='relu', strides=1)(c1)
max = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same')(conv_2)
fc_1 = Flatten()(max)
out = Dense(256)(fc_1)
heads.append(out)
fc_1_drop = RepeatVector(1)(out)
out_lstm = LSTM(64, return_sequences=False)(fc_1_drop)
out_lstm = Reshape((64,))(out_lstm)
heads.append(out_lstm)
out_cnnlstm= Concatenate()(heads)
towers.append(out_cnnlstm)
y = Model(inputs=inp_y, outputs=out_cnnlstm)
heads = []
#
c1 = Conv2D(30, (5,5), padding='same', activation='relu', strides=1)(inp_z)
conv_2 = Conv2D(20, (5,5), padding='same', activation='relu', strides=1)(c1)
max = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same')(conv_2)
fc_1 = Flatten()(max)
out = Dense(256)(fc_1)
heads.append(out)
fc_1_drop = RepeatVector(1)(out)
out_lstm = LSTM(64, return_sequences=False)(fc_1_drop)
out_lstm = Reshape((64,))(out_lstm)
heads.append(out_lstm)
out_cnnlstm= Concatenate()(heads)
towers.append(out_cnnlstm)
z = Model(inputs=inp_z, outputs=out_cnnlstm)
heads = []
#
out = Concatenate()(towers)
out = Reshape((960,))(out)
out = RepeatVector(1)(out)
d = LSTM(64, return_sequences=False)(out)
d = Dense(128)(d)
d_out = Dense(2, activation='softmax')(d)
d_out = Reshape((2,))(d_out)
cnn2lstm_model = Model(inputs=[inp_x, inp_y, inp_z], outputs=d_out)
cnn2lstm_model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
history_CNN2LSTM = cnn2lstm_model.fit([Xcx_trainx, Xcy_trainx, Xcz_trainx], Yc_trainx, batch_size=100, epochs=10, verbose=1, validation_data=([Xcx_testx, Xcy_testx, Xcz_testx], Yc_testx))
cnn2lstm_model.save('/content/gdrive/My Drive/Cooperative_LSTM.h5')

array([   0,  256,  512,  768, 1024])